# 主表处理

In [1]:
import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import gc
import warnings

# 数据处理包
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
warnings.filterwarnings("ignore")
path = "c:/data/"

In [2]:
def reduce_mem_usage(data, verbose = True):
    start_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for col in data.columns:
        col_type = data[col].dtype
        
        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)

    end_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return data

In [5]:
# application = pd.read_csv("appIication_train.csv",index_col="SK_ID_CURR")

application = pd.read_csv("appIication_train.csv")
application = reduce_mem_usage(application)
application.shape

Memory usage of dataframe: 286.23 MB
Memory usage after optimization: 92.38 MB
Decreased by 67.7%


(307511, 122)

In [11]:
pd.set_option('display.max_columns', None) #让pandas显示所有列
application.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018799,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.262939,0.024704,0.036896,0.972168,0.619141,0.014297,0.000000,0.068970,0.083313,0.125000,0.036896,0.020203,0.018997,0.000000,0.000000,0.025208,0.038300,0.972168,0.634277,0.014397,0.000000,0.068970,0.083313,0.125000,0.037689,0.022003,0.019806,0.0,0.0,0.024994,0.036896,0.972168,0.624512,0.014397,0.000000,0.068970,0.083313,0.125000,0.037506,0.020493,0.019302,0.000000,0.000000,reg oper account,block of flats,0.014900,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.067322,0.498047
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003542,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.622070,0.095886,0.052887,0.984863,0.795898,0.060486,0.080017,0.034485,0.291748,0.333252,0.013000,0.077271,0.054901,0.003901,0.009804,0.092407,0.053802,0.984863,0.804199,0.049713,0.080627,0.034485,0.291748,0.333252,0.012802,0.078979,0.055389,0.0,0.0,0.096802,0.052887,0.984863,0.798828,0.060791,0.080017,0.034485,0.291748,0.333252,0.013199,0.078674,0.055786,0.003901,0.010002,reg oper account,block of flats,0.071411,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.070862,0.208740
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010033,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,

In [16]:
application["DAYS_EMPLOYED"].max()

365243

In [19]:
temp = application[["SK_ID_CURR","TARGET","NAME_EDUCATION_TYPE",
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'DAYS_EMPLOYED_PERC',
 'INCOME_CREDIT_PERC']]

In [20]:
temp.head(20)

,SK_ID_CURR,TARGET,NAME_EDUCATION_TYPE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC
0,100002,1,Secondary / secondary special,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.067322,0.498047
1,100003,0,Higher education,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.070862,0.208740
2,100004,0,Secondary / secondary special,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.011818,0.500000
3,100006,0,Secondary / secondary special,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.159912,0.431641
4,100007,0,Secondary / secondary special,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.152466,0.236816
5,100008,0,Secondary / secondary special,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.093750,0.201782
6,100009,0,Higher education,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0,0.227173,0.109558
7,100010,0,Higher education,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.023819,0.235352
8,100011,0,Secondary / secondary special,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,-18.171875,0.110352
9,100012,0,Secondary / secondary special,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.139526,0.333252


In [21]:
application.columns.tolist()

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_2',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONA

In [22]:
for col in temp.columns:
    print("\n %s 属性的不同取值和出现的次数" %col)
    print(temp[col].value_counts())


 SK_ID_CURR 属性的不同取值和出现的次数
100002    1
337664    1
337661    1
337660    1
337659    1
         ..
218992    1
218991    1
218990    1
218989    1
456255    1
Name: SK_ID_CURR, Length: 307511, dtype: int64

 TARGET 属性的不同取值和出现的次数
0    282686
1     24825
Name: TARGET, dtype: int64

 NAME_EDUCATION_TYPE 属性的不同取值和出现的次数
Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: NAME_EDUCATION_TYPE, dtype: int64

 FLAG_DOCUMENT_2 属性的不同取值和出现的次数
0    307498
1        13
Name: FLAG_DOCUMENT_2, dtype: int64

 FLAG_DOCUMENT_3 属性的不同取值和出现的次数
1    218340
0     89171
Name: FLAG_DOCUMENT_3, dtype: int64

 FLAG_DOCUMENT_4 属性的不同取值和出现的次数
0    307486
1        25
Name: FLAG_DOCUMENT_4, dtype: int64

 FLAG_DOCUMENT_5 属性的不同取值和出现的次数
0    302863
1      4648
Name: FLAG_DOCUMENT_5, dtype: int64

 FLAG_DOCUMENT_6 属性的不同取值和出现的次数
0    280433
1     27078
Name: FLAG_DOCUMENT_

In [39]:
sk_id = application.index.tolist()

In [40]:
sk_id

[100002,
 100003,
 100004,
 100006,
 100007,
 100008,
 100009,
 100010,
 100011,
 100012,
 100014,
 100015,
 100016,
 100017,
 100018,
 100019,
 100020,
 100021,
 100022,
 100023,
 100024,
 100025,
 100026,
 100027,
 100029,
 100030,
 100031,
 100032,
 100033,
 100034,
 100035,
 100036,
 100037,
 100039,
 100040,
 100041,
 100043,
 100044,
 100045,
 100046,
 100047,
 100048,
 100049,
 100050,
 100051,
 100052,
 100053,
 100054,
 100055,
 100056,
 100058,
 100059,
 100060,
 100061,
 100062,
 100063,
 100064,
 100068,
 100069,
 100070,
 100071,
 100072,
 100073,
 100075,
 100076,
 100077,
 100078,
 100079,
 100080,
 100081,
 100082,
 100083,
 100084,
 100085,
 100086,
 100087,
 100088,
 100089,
 100093,
 100094,
 100095,
 100096,
 100097,
 100098,
 100099,
 100100,
 100101,
 100102,
 100103,
 100104,
 100105,
 100108,
 100110,
 100111,
 100112,
 100113,
 100114,
 100115,
 100116,
 100118,
 100119,
 100120,
 100121,
 100122,
 100123,
 100124,
 100125,
 100126,
 100127,
 100129,
 100130,
 

In [41]:
application['DAYS_EMPLOYED'] = (application['DAYS_EMPLOYED'].apply(lambda x: x if x != 365243 else 0))

# 合成新特征

In [42]:
application['DAYS_EMPLOYED_PERC'] = application['DAYS_EMPLOYED'] / (application['DAYS_BIRTH'] + 1)
application['INCOME_CREDIT_PERC'] = application['AMT_INCOME_TOTAL'] / (application['AMT_CREDIT'] +1)# 收入/贷款
application['INCOME_PER_PERSON'] = application['AMT_INCOME_TOTAL'] / (application['CNT_FAM_MEMBERS']) # 人均收入
application['ANNUITY_INCOME_PERC'] = application['AMT_ANNUITY'] / (application['AMT_INCOME_TOTAL']+1)  # 贷款百分比
application['PAYMENT_RATE'] = application['AMT_ANNUITY'] / (application['AMT_CREDIT']+1)
docs = [_f for _f in application.columns if 'FLAG_DOC' in _f]
live = [_f for _f in application.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
inc_by_org = application[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
application['NEW_CREDIT_TO_ANNUITY_RATIO'] = application['AMT_CREDIT'] / (application['AMT_ANNUITY']+1)
application['NEW_CREDIT_TO_GOODS_RATIO'] = application['AMT_CREDIT'] / (application['AMT_GOODS_PRICE']+1)
application['NEW_DOC_IND_AVG'] = application[docs].mean(axis=1)
application['NEW_DOC_IND_STD'] = application[docs].std(axis=1)
application['NEW_DOC_IND_KURT'] = application[docs].kurtosis(axis=1) # kurtosis是用在概率论和统计学里。它描述了例如频率分布的尖峰形式
application['NEW_LIVE_IND_SUM'] = application[live].sum(axis=1)
application['NEW_LIVE_IND_STD'] = application[live].std(axis=1)
application['NEW_LIVE_IND_KURT'] = application[live].kurtosis(axis=1)
application['NEW_INC_PER_CHLD'] = application['AMT_INCOME_TOTAL'] / (1 + application['CNT_CHILDREN'])
application['NEW_INC_BY_ORG'] = application['ORGANIZATION_TYPE'].map(inc_by_org)
application['NEW_CAR_TO_BIRTH_RATIO'] = application['OWN_CAR_AGE'] / (application['DAYS_BIRTH']+1)
application['NEW_CAR_TO_EMPLOY_RATIO'] = application['OWN_CAR_AGE'] / (application['DAYS_EMPLOYED']+1)
application['NEW_PHONE_TO_BIRTH_RATIO'] = application['DAYS_LAST_PHONE_CHANGE'] / (application['DAYS_BIRTH']+1)
application['NEW_PHONE_TO_EMPLOY_RATIO'] = application['DAYS_LAST_PHONE_CHANGE'] / (application['DAYS_EMPLOYED']+1)

In [43]:
# 删除缺失值较多的特征
def missing_values(df):
        # 缺失的总个数
        mis_val = df.isnull().sum()
        
        # 计算百分比
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # 合并
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # 重命名
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Count', 1 : 'percent'})
        
        # 根据缺失值降序排列
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('percent', ascending=False).round(1)
        
        # Print some summary information
        print ("selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [44]:
missing_values(application).head(20)

selected dataframe has 138 columns.
There are 74 columns that have missing values.


,Missing Count,percent
COMMONAREA_MEDI,214865,69.9
COMMONAREA_MODE,214865,69.9
COMMONAREA_AVG,214865,69.9
NONLIVINGAPARTMENTS_MODE,213514,69.4
NONLIVINGAPARTMENTS_MEDI,213514,69.4
NONLIVINGAPARTMENTS_AVG,213514,69.4
FONDKAPREMONT_MODE,210295,68.4
LIVINGAPARTMENTS_MEDI,210199,68.4
LIVINGAPARTMENTS_MODE,210199,68.4
LIVINGAPARTMENTS_AVG,210199,68.4


In [45]:
application.shape

(307511, 138)

# 删除缺失值

In [46]:
def drop_columns(df,percent):
    # 删除超过百分比的列
    # 还要加上table, percent参数
    table = missing_values(df)
    drop_list = table.loc[table["percent"] > percent,:].index.tolist()
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [47]:
drop_columns(application, 60)

selected dataframe has 138 columns.
There are 74 columns that have missing values.


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,NEW_DOC_IND_AVG,NEW_DOC_IND_STD,NEW_DOC_IND_KURT,NEW_LIVE_IND_SUM,NEW_LIVE_IND_STD,NEW_LIVE_IND_KURT,NEW_INC_PER_CHLD,NEW_INC_BY_ORG,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_EMPLOY_RATIO
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0.05,0.223607,20.0,4,0.516398,-1.875000,202500.0,157500.0,0.119873,1.783019
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0.05,0.223607,20.0,4,0.516398,-1.875000,270000.0,135000.0,0.049392,0.697557
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0.00,0.000000,0.0,5,0.408248,6.000000,67500.0,135000.0,0.042793,3.638393
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0.05,0.223607,20.0,3,0.547723,-3.333333,135000.0,157500.0,0.032467,0.203094
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0.05,0.223607,20.0,3,0.547723,-3.333333,121500.0,162000.0,0.055491,0.364175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,...,0.05,0.223607,20.0,3,0.547723,-3.333333,157500.0,157500.0,0.029273,1.161702
456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,...,0.05,0.223607,20.0,3,0.547723,-3.333333,72000.0,117000.0,-0.000000,0.000000
456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,...,0.05,0.223607,20.0,4,0.516398,-1.875000,153000.0,135000.0,0.127564,0.241035


In [48]:
application.shape

(307511, 119)

# 修改异常值

In [49]:
application['DAYS_EMPLOYED'].value_counts()

 0        55376
-200        156
-224        152
-230        151
-199        151
          ...  
-12588        1
-8229         1
-12794        1
-11823        1
-8694         1
Name: DAYS_EMPLOYED, Length: 12573, dtype: int64

In [50]:
application['DAYS_EMPLOYED'] = (application['DAYS_EMPLOYED'].apply(lambda x: x if x != 365243 else 0))

# 同值化处理

In [51]:
for col in application.columns:
    print("\n %s 属性的不同取值和出现的次数" %col)
    print(application[col].value_counts())


 TARGET 属性的不同取值和出现的次数
0    282686
1     24825
Name: TARGET, dtype: int64

 NAME_CONTRACT_TYPE 属性的不同取值和出现的次数
Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64

 CODE_GENDER 属性的不同取值和出现的次数
F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64

 FLAG_OWN_CAR 属性的不同取值和出现的次数
N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64

 FLAG_OWN_REALTY 属性的不同取值和出现的次数
Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64

 CNT_CHILDREN 属性的不同取值和出现的次数
0     215371
1      61119
2      26749
3       3717
4        429
5         84
6         21
7          7
14         3
8          2
9          2
12         2
10         2
19         2
11         1
Name: CNT_CHILDREN, dtype: int64

 AMT_INCOME_TOTAL 属性的不同取值和出现的次数
135000.0    35750
112500.0    31019
157500.0    26556
180000.0    24719
90000.0     22483
            ...  
117324.0        1
64584.0         1
142897.5        1
109170.0        1
113062.5        1
Name: AMT_INCOME_TOTAL, Length: 

0.000000    60954
0.001200      539
0.002199      478
0.003700      456
0.004398      431
            ...  
0.247314        1
0.430908        1
0.203003        1
0.418701        1
0.293945        1
Name: NONLIVINGAREA_MEDI, Length: 2923, dtype: int64

 HOUSETYPE_MODE 属性的不同取值和出现的次数
block of flats      150503
specific housing      1499
terraced house        1212
Name: HOUSETYPE_MODE, dtype: int64

 TOTALAREA_MODE 属性的不同取值和出现的次数
0.000000    582
0.057007    247
0.054688    230
0.054810    227
0.054993    227
           ... 
0.459717      1
0.477295      1
0.734375      1
0.460693      1
0.796875      1
Name: TOTALAREA_MODE, Length: 3660, dtype: int64

 WALLSMATERIAL_MODE 属性的不同取值和出现的次数
Panel           66040
Stone, brick    64815
Block            9253
Wooden           5362
Mixed            2296
Monolithic       1779
Others           1625
Name: WALLSMATERIAL_MODE, dtype: int64

 EMERGENCYSTATE_MODE 属性的不同取值和出现的次数
No     159428
Yes      2328
Name: EMERGENCYSTATE_MODE, dtype: int64

 OBS_30_CNT_S

In [52]:
# 删除同值化超过95%的特征

drop_list = ["FLAG_MOBIL","FLAG_CONT_MOBILE","REG_REGION_NOT_LIVE_REGION","LIVE_REGION_NOT_WORK_REGION","FLAG_DOCUMENT_2","FLAG_DOCUMENT_4",
       "FLAG_DOCUMENT_5","FLAG_DOCUMENT_7","FLAG_DOCUMENT_9","FLAG_DOCUMENT_10","FLAG_DOCUMENT_11","FLAG_DOCUMENT_12","FLAG_DOCUMENT_13",
       "FLAG_DOCUMENT_14","FLAG_DOCUMENT_15","FLAG_DOCUMENT_16","FLAG_DOCUMENT_17","FLAG_DOCUMENT_18","FLAG_DOCUMENT_19","FLAG_DOCUMENT_20",
       "FLAG_DOCUMENT_21"]

In [53]:
application.drop(labels = drop_list,axis = 1, inplace = True)

In [54]:
application.shape

(307511, 98)

# 编码

In [55]:
def encoder(df):
    original_columns = list(df.columns)
    # Create a label encoder object
    le = LabelEncoder()
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])
    
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= True) # 空也算一种类型
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def data_labelencode(data):
    # 创建一个Label对象
    le = LabelEncoder()
    le_count = 0

    # 遍历每一个变量
    for col in data:
        if data[col].dtype == 'object':
            # 特征内只有两个变量
            if len(list(data[col].unique())) <= 2:
                # 编码
                le.fit(data[col])
                # 编码
                data[col] = le.transform(data[col])

                # 统计编码个数
                le_count += 1

    print('%d columns were label encoded.' % le_count)
    return data


def encode_impute(df):
    df = data_labelencode(df)
    df = pd.get_dummies(df)
    types = np.array([z for z in df.dtypes])
    all_columns = df.columns.values               
    is_num = types != 'object'  
    #is_num = np.array([str(x)[:1] != 'c' for x in types]) 
    num_features = all_columns[is_num].tolist()         
    cat_features = all_columns[~is_num].tolist() 
    features = num_features + cat_features
    Pipe_num = Pipeline(
        steps=[
        ('imputer', SimpleImputer(strategy = 'median')),        # tried median, mean, constant strategies
        ('scaler', StandardScaler())       ])

    Pipe_cat = Pipeline(
        steps=[
        ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'Unknown')),
        ('onehot', OneHotEncoder())])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', Pipe_num, num_features),
            ('cat', Pipe_cat, cat_features)])

    data = preprocessor.fit_transform(df[features])
    return data, df.columns

In [56]:
data_X = application.drop(['TARGET'], axis=1)  # X dataFrame
y = application["TARGET"]  # Series
data_X.replace([np.inf, -np.inf], np.nan, inplace=True)
data_X.fillna(0, inplace=True)
data, columns = encode_impute(data_X)
data_app =  pd.DataFrame(data, columns=columns,index = sk_id)
data_app = pd.concat([data_app, y], axis=1)
data_app.index.name = 'SK_ID_CURR'

3 columns were label encoded.


In [57]:
data_app.head(10)

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_0,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-0.324395,-0.717914,0.664531,-0.577538,0.142129,-0.478095,-0.166065,-0.505662,-0.149689,1.506880,...,-0.086733,-0.076281,-0.072886,-0.522963,1.935056,-0.133215,-0.949252,0.963763,-0.08734,1
100003,-0.324395,-0.717914,-1.504820,-0.577538,0.426792,1.725450,0.592710,1.600480,-1.252595,-0.166821,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,-0.949252,0.963763,-0.08734,0
100004,3.082659,1.392925,0.664531,-0.577538,-0.427196,-1.152888,-1.404507,-1.090025,-0.783388,-0.689509,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100006,-0.324395,-0.717914,0.664531,-0.577538,-0.142533,-0.711430,0.177929,-0.651753,-0.928990,-0.680114,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100007,-0.324395,-0.717914,0.664531,-0.577538,-0.199466,-0.213734,-0.361658,-0.067390,0.562878,-0.892535,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100008,-0.324395,-0.717914,0.664531,-0.577538,-0.294354,-0.269648,0.028286,-0.225655,1.079103,-0.207151,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100009,-0.324395,1.392925,0.664531,0.807273,0.009287,2.389375,0.979238,2.318760,1.079103,0.517645,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100010,-0.324395,1.392925,0.664531,-0.577538,0.806342,2.313036,1.032638,2.683987,-1.282929,-0.644596,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100011,-0.324395,-0.717914,0.664531,-0.577538,-0.237421,1.044955,0.463557,1.016117,-0.161822,-0.930803,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0


In [58]:
data_app.shape

(307511, 221)

In [59]:
data_app.head(10)

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_0,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-0.324395,-0.717914,0.664531,-0.577538,0.142129,-0.478095,-0.166065,-0.505662,-0.149689,1.506880,...,-0.086733,-0.076281,-0.072886,-0.522963,1.935056,-0.133215,-0.949252,0.963763,-0.08734,1
100003,-0.324395,-0.717914,-1.504820,-0.577538,0.426792,1.725450,0.592710,1.600480,-1.252595,-0.166821,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,-0.949252,0.963763,-0.08734,0
100004,3.082659,1.392925,0.664531,-0.577538,-0.427196,-1.152888,-1.404507,-1.090025,-0.783388,-0.689509,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100006,-0.324395,-0.717914,0.664531,-0.577538,-0.142533,-0.711430,0.177929,-0.651753,-0.928990,-0.680114,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100007,-0.324395,-0.717914,0.664531,-0.577538,-0.199466,-0.213734,-0.361658,-0.067390,0.562878,-0.892535,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100008,-0.324395,-0.717914,0.664531,-0.577538,-0.294354,-0.269648,0.028286,-0.225655,1.079103,-0.207151,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100009,-0.324395,1.392925,0.664531,0.807273,0.009287,2.389375,0.979238,2.318760,1.079103,0.517645,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100010,-0.324395,1.392925,0.664531,-0.577538,0.806342,2.313036,1.032638,2.683987,-1.282929,-0.644596,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0
100011,-0.324395,-0.717914,0.664531,-0.577538,-0.237421,1.044955,0.463557,1.016117,-0.161822,-0.930803,...,-0.086733,-0.076281,-0.072886,-0.522963,-0.516781,-0.133215,1.053461,-1.037599,-0.08734,0


In [60]:
data_app.to_csv("C:/data/processing_data/data_app.csv")

# 读取其他文件

# bureau

In [27]:
# 总结处理过程
def data_processing(df):
    # 正无穷和负无穷使用nan填充
    bureau.replace([np.inf, -np.inf], np.nan, inplace=True)
    # nan使用0填充
    bureau.fillna(0, inplace=True)
    print("before process df shape:",df.shape)
    print(missing_values(df).head(30))
    print("drop missing values over 60")
    drop_columns(df, 60)
    print("after drop missing values df shape:",df.shape)
    drop_list = []
    # 同值化处理，删除超过特征中某个值个数超过0.95的特征
    for col in df.columns:
        # print("\n %s 属性的不同取值和出现的次数" %col)
        #  print(df[col].value_counts())
        if df[col].value_counts().max() > df.shape[0]*0.95:
            drop_list.append(col)
    print("the same values feature num :", len(drop_list))
    df.drop(labels = drop_list, axis = 1, inplace = True)
    print("after process df shape:",df.shape)
    return df

In [28]:
b = pd.read_csv(path + 'bureau.csv')
b

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [29]:
b = pd.read_csv(path + 'bureau.csv')
b.head(20)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN


In [30]:
# bureau and bureau_balance
def bureau_and_balance(num_rows = None):
    bureau = pd.read_csv(path + 'bureau.csv', nrows = num_rows)
    bureau = reduce_mem_usage(bureau)
    bb = pd.read_csv(path+ 'bureau_balance.csv', nrows = num_rows)
    bb = reduce_mem_usage(bb)
    print("before bureau shape:", bureau.shape)
    print("before bureau balance shape:", bb.shape)
    bb, bb_cat = encoder(bb)
    bureau, bureau_cat = encoder(bureau)
    
    # 同一个用户有多个记录，进行计算平均值
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    print(bb_agg.head(10))
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()]) # 列明
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # 聚合特征
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # 类别特征
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
        
    # **形参是字典类型
    
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()]) # 聚合的名字规则，列名+类别名称
    # Bureau: Active credits - 数值类型，active为1的聚合
    print(bureau_agg.head(10))
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print(bureau.head(10))
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    cols = active_agg.columns.tolist()
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()]) 
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - 已经关闭的贷款，
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    for e in cols:
        bureau_agg['NEW_RATIO_BURO_' + e[0] + "_" + e[1].upper()] = bureau_agg['ACTIVE_' + e[0] + "_" + e[1].upper()] / bureau_agg['CLOSED_' + e[0] + "_" + e[1].upper()]
    
    del closed, closed_agg, bureau
    gc.collect()
    print("after shape", bureau_agg.shape)
    return bureau_agg


In [31]:
bureau = bureau_and_balance()

Memory usage of dataframe: 222.62 MB
Memory usage after optimization: 112.95 MB
Decreased by 49.3%
Memory usage of dataframe: 624.85 MB
Memory usage after optimization: 338.46 MB
Decreased by 45.8%
before bureau shape: (1716428, 17)
before bureau balance shape: (27299925, 3)
             MONTHS_BALANCE           STATUS_0  STATUS_1 STATUS_2 STATUS_3  \
                        min max size      mean      mean     mean     mean   
SK_ID_BUREAU                                                                 
5001709                 -96   0   97  0.000000  0.000000      0.0      0.0   
5001710                 -82   0   83  0.060241  0.000000      0.0      0.0   
5001711                  -3   0    4  0.750000  0.000000      0.0      0.0   
5001712                 -18   0   19  0.526316  0.000000      0.0      0.0   
5001713                 -21   0   22  0.000000  0.000000      0.0      0.0   
5001714                 -14   0   15  0.000000  0.000000      0.0      0.0   
5001715               

after shape (305811, 143)


In [32]:
bureau.shape

(305811, 143)

In [33]:
missing_values(bureau).head(30)

selected dataframe has 143 columns.
There are 105 columns that have missing values.


,Missing Count,percent
NEW_RATIO_BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,302523,98.9
NEW_RATIO_BURO_CREDIT_DAY_OVERDUE_MEAN,302517,98.9
NEW_RATIO_BURO_CREDIT_DAY_OVERDUE_MAX,302517,98.9
NEW_RATIO_BURO_CNT_CREDIT_PROLONG_SUM,298116,97.5
NEW_RATIO_BURO_MONTHS_BALANCE_MAX_MAX,292001,95.5
NEW_RATIO_BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,259607,84.9
NEW_RATIO_BURO_AMT_CREDIT_SUM_LIMIT_MEAN,258988,84.7
NEW_RATIO_BURO_AMT_CREDIT_SUM_LIMIT_SUM,254729,83.3
NEW_RATIO_BURO_AMT_ANNUITY_MEAN,245109,80.2
NEW_RATIO_BURO_AMT_ANNUITY_MAX,245109,80.2


In [34]:
bureau.shape

(305811, 143)

In [35]:
drop_columns(bureau, 60).head(20)

selected dataframe has 143 columns.
There are 105 columns that have missing values.


,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MIN,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MAX,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MEAN,NEW_RATIO_BURO_DAYS_CREDIT_UPDATE_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_MAX,NEW_RATIO_BURO_AMT_CREDIT_SUM_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_SUM,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_MAX,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,2.400437e+05,-1329.0,1778.0,82.4375,-93.142857,0,0.0,...,-0.309326,-9.929688,-1.638672,0.068817,1.351351,2.070292,1.552719,inf,inf,inf
100002,-1437,-103,-874.000000,1.861500e+05,-1072.0,780.0,-349.0000,-499.875000,0,0.0,...,-0.727539,9.179688,-1.356445,0.023438,3.333333,3.774707,1.258236,inf,inf,inf
100003,-2586,-606,-1400.750000,8.277836e+05,-2434.0,1216.0,-544.5000,-816.000000,0,0.0,...,-0.499512,-2.894531,-1.075195,0.040050,7.200000,11.716461,3.905487,NaN,NaN,NaN
100004,-1326,-408,-867.000000,4.213620e+05,-595.0,-382.0,-488.5000,-532.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-373,-62,-190.666667,2.634033e+04,-128.0,1324.0,439.2500,-54.333333,0,0.0,...,-0.953125,-10.343750,-5.648438,0.173554,9.723077,5.116462,10.232924,inf,inf,inf
100007,-1149,-1149,-1149.000000,NaN,-783.0,-783.0,-783.0000,-783.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100008,-1097,-78,-757.333333,3.461203e+05,-853.0,471.0,-391.2500,-611.000000,0,0.0,...,-0.552246,-0.594727,-0.572754,0.017611,2.531631,2.664874,1.332437,inf,inf,inf
100009,-2882,-239,-1271.500000,4.426300e+05,-2152.0,1402.0,-795.0000,-851.611111,0,0.0,...,0.097107,1.342773,-0.600586,0.239823,0.361666,1.633381,0.466680,inf,inf,inf
100010,-2741,-1138,-1939.500000,1.284804e+06,-928.0,689.0,-119.5000,-578.000000,0,0.0,...,-0.742676,-0.742676,-0.742676,0.015817,2.142857,2.142857,2.142857,inf,inf,inf


In [36]:
bureau.shape

(305811, 118)

In [37]:
bureau.replace([np.inf, -np.inf], np.nan, inplace=True)
bureau.fillna(0, inplace=True)

In [38]:
bureau.head(10)

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MIN,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MAX,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MEAN,NEW_RATIO_BURO_DAYS_CREDIT_UPDATE_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_MAX,NEW_RATIO_BURO_AMT_CREDIT_SUM_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_SUM,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_MAX,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_DEBT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,2.400437e+05,-1329.0,1778.0,82.4375,-93.142857,0,0.0,...,-0.309326,-9.929688,-1.638672,0.068817,1.351351,2.070292,1.552719,0.0,0.0,0.0
100002,-1437,-103,-874.000000,1.861500e+05,-1072.0,780.0,-349.0000,-499.875000,0,0.0,...,-0.727539,9.179688,-1.356445,0.023438,3.333333,3.774707,1.258236,0.0,0.0,0.0
100003,-2586,-606,-1400.750000,8.277836e+05,-2434.0,1216.0,-544.5000,-816.000000,0,0.0,...,-0.499512,-2.894531,-1.075195,0.040050,7.200000,11.716461,3.905487,0.0,0.0,0.0
100004,-1326,-408,-867.000000,4.213620e+05,-595.0,-382.0,-488.5000,-532.000000,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
100005,-373,-62,-190.666667,2.634033e+04,-128.0,1324.0,439.2500,-54.333333,0,0.0,...,-0.953125,-10.343750,-5.648438,0.173554,9.723077,5.116462,10.232924,0.0,0.0,0.0
100007,-1149,-1149,-1149.000000,0.000000e+00,-783.0,-783.0,-783.0000,-783.000000,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
100008,-1097,-78,-757.333333,3.461203e+05,-853.0,471.0,-391.2500,-611.000000,0,0.0,...,-0.552246,-0.594727,-0.572754,0.017611,2.531631,2.664874,1.332437,0.0,0.0,0.0
100009,-2882,-239,-1271.500000,4.426300e+05,-2152.0,1402.0,-795.0000,-851.611111,0,0.0,...,0.097107,1.342773,-0.600586,0.239823,0.361666,1.633381,0.466680,0.0,0.0,0.0
100010,-2741,-1138,-1939.500000,1.284804e+06,-928.0,689.0,-119.5000,-578.000000,0,0.0,...,-0.742676,-0.742676,-0.742676,0.015817,2.142857,2.142857,2.142857,0.0,0.0,0.0


# 同值化处理

In [39]:
drop_list_bureau = []
for col in bureau.columns:
    print("\n %s 属性的不同取值和出现的次数" %col)
    print(bureau[col].value_counts())
    if bureau[col].value_counts().max() > 290520:
        drop_list_bureau.append(col)


 BURO_DAYS_CREDIT_MIN 属性的不同取值和出现的次数
-2871    323
-2919    322
-2890    319
-2895    315
-2891    311
        ... 
-5         6
-4         4
 0         2
-2         1
-3         1
Name: BURO_DAYS_CREDIT_MIN, Length: 2922, dtype: int64

 BURO_DAYS_CREDIT_MAX 属性的不同取值和出现的次数
-91      751
-182     695
-175     689
-112     688
-84      680
        ... 
-2838      2
-2137      2
-1962      2
-2010      2
-2043      1
Name: BURO_DAYS_CREDIT_MAX, Length: 2923, dtype: int64

 BURO_DAYS_CREDIT_MEAN 属性的不同取值和出现的次数
-441.000000     94
-400.000000     86
-478.000000     86
-809.000000     83
-488.000000     81
                ..
-1636.437500     1
-792.714286      1
-1030.555556     1
-730.875000      1
-1667.076923     1
Name: BURO_DAYS_CREDIT_MEAN, Length: 69801, dtype: int64

 BURO_DAYS_CREDIT_VAR 属性的不同取值和出现的次数
0.000000         43739
0.500000          1081
4.500000           192
2.000000           162
18.000000          123
                 ...  
635554.322222        1
708402.982456        1
24224

Name: BURO_CREDIT_TYPE_Real estate loan_MEAN, dtype: int64

 BURO_CREDIT_TYPE_Unknown type of loan_MEAN 属性的不同取值和出现的次数
0.000000    305314
0.166667        51
0.142857        46
0.111111        41
0.125000        40
0.250000        37
0.200000        34
0.333333        33
0.090909        28
0.500000        26
0.100000        26
0.076923        19
0.058824        13
0.083333        13
0.071429        12
0.066667        10
0.050000         7
0.062500         6
1.000000         6
0.055556         5
0.052632         5
0.105263         4
0.047619         3
0.086957         3
0.222222         2
0.095238         2
0.176471         2
0.117647         2
0.037037         2
0.181818         2
0.041667         2
0.400000         2
0.133333         2
0.666667         1
0.157895         1
0.187500         1
0.051282         1
0.043478         1
0.040000         1
0.285714         1
0.153846         1
0.029412         1
0.428571         1
0.035714         1
Name: BURO_CREDIT_TYPE_Unknown type of loan_ME


 ACTIVE_AMT_CREDIT_SUM_DEBT_MAX 属性的不同取值和出现的次数
0.0          91282
225000.0       144
4.5            132
135000.0       129
450000.0       123
             ...  
1069501.5        1
384646.5         1
1040566.5        1
77841.0          1
595102.5         1
Name: ACTIVE_AMT_CREDIT_SUM_DEBT_MAX, Length: 156640, dtype: int64

 ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN 属性的不同取值和出现的次数
0.000000e+00    90734
4.500000e+00       88
2.250000e+05       68
4.500000e+05       55
4.500000e+04       54
                ...  
1.863755e+05        1
4.042575e+05        1
3.491398e+05        1
1.051844e+06        1
3.069826e+05        1
Name: ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN, Length: 181798, dtype: int64

 ACTIVE_AMT_CREDIT_SUM_DEBT_SUM 属性的不同取值和出现的次数
0.000000e+00    90734
4.500000e+00      130
2.250000e+05       66
4.500000e+05       62
1.350000e+05       60
                ...  
6.857100e+04        1
1.109267e+07        1
8.445618e+04        1
1.130580e+06        1
1.534913e+06        1
Name: ACTIVE_AMT_CREDIT_SUM_DE


 NEW_RATIO_BURO_DAYS_CREDIT_MIN 属性的不同取值和出现的次数
0.000000    91696
1.000000     4206
0.500000       63
0.333333       43
0.200000       40
            ...  
0.101595        1
2.548822        1
0.233942        1
0.514816        1
0.436457        1
Name: NEW_RATIO_BURO_DAYS_CREDIT_MIN, Length: 190453, dtype: int64

 NEW_RATIO_BURO_DAYS_CREDIT_MAX 属性的不同取值和出现的次数
0.000000    91714
1.000000     4214
0.500000      215
0.333333      175
0.250000      115
            ...  
0.438580        1
0.822196        1
0.990783        1
0.470699        1
0.374227        1
Name: NEW_RATIO_BURO_DAYS_CREDIT_MAX, Length: 152058, dtype: int64

 NEW_RATIO_BURO_DAYS_CREDIT_MEAN 属性的不同取值和出现的次数
0.000000    91696
1.000000      719
0.500000       38
0.333333       23
0.250000       23
            ...  
0.623569        1
0.265757        1
0.088881        1
0.187132        1
0.363300        1
Name: NEW_RATIO_BURO_DAYS_CREDIT_MEAN, Length: 207074, dtype: int64

 NEW_RATIO_BURO_DAYS_CREDIT_VAR 属性的不同取值和出现的次数
0.000000     18

In [40]:
bureau.shape[0] 

305811

In [41]:
bureau.BURO_AMT_CREDIT_SUM_SUM.value_counts().max()

1513

In [42]:
len(drop_list_bureau)

46

In [43]:
bureau.shape

(305811, 118)

In [44]:
bureau.drop(labels = drop_list_bureau, axis = 1, inplace = True)

In [45]:
bureau.shape

(305811, 72)

In [46]:
bureau.to_csv("C:/data/processing_data/bureau.csv")

In [47]:
bureau

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_MAX,...,NEW_RATIO_BURO_DAYS_CREDIT_MAX,NEW_RATIO_BURO_DAYS_CREDIT_MEAN,NEW_RATIO_BURO_DAYS_CREDIT_VAR,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MIN,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MAX,NEW_RATIO_BURO_DAYS_CREDIT_ENDDATE_MEAN,NEW_RATIO_BURO_DAYS_CREDIT_UPDATE_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_MAX,NEW_RATIO_BURO_AMT_CREDIT_SUM_MEAN,NEW_RATIO_BURO_AMT_CREDIT_SUM_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.4375,-93.142857,0.000000,3.780000e+05,...,0.057176,0.293416,0.544427,-0.309326,-9.929688,-1.638672,0.068817,1.351351,2.070292,1.552719
100002,-1437,-103,-874.000000,186150.000000,-1072.0,780.0,-349.0000,-499.875000,1681.029053,4.500000e+05,...,0.216387,0.587481,3.556569,-0.727539,9.179688,-1.356445,0.023438,3.333333,3.774707,1.258236
100003,-2586,-606,-1400.750000,827783.583333,-2434.0,1216.0,-544.5000,-816.000000,0.000000,8.100000e+05,...,0.781935,0.363818,0.000000,-0.499512,-2.894531,-1.075195,0.040050,7.200000,11.716461,3.905487
100004,-1326,-408,-867.000000,421362.000000,-595.0,-382.0,-488.5000,-532.000000,0.000000,9.453780e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100005,-373,-62,-190.666667,26340.333333,-128.0,1324.0,439.2500,-54.333333,0.000000,5.688000e+05,...,0.166220,0.266756,0.000000,-0.953125,-10.343750,-5.648438,0.173554,9.723077,5.116462,10.232924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-1667.076923,407302.243590,-2500.0,1363.0,-1232.0000,-1064.538462,6147.000000,7.650000e+05,...,0.534884,0.916321,8.758694,0.598633,-2.072266,0.045715,0.534129,0.294118,0.677287,0.123143
456250,-1002,-760,-862.000000,15724.000000,-272.0,2340.0,1288.0000,-60.333333,0.000000,2.153110e+06,...,0.758483,0.790419,0.000000,-6.605469,-8.601562,-7.601562,0.212598,4.454561,2.692782,5.385565
456253,-919,-713,-867.500000,10609.000000,-189.0,1113.0,280.5000,-253.250000,0.000000,2.250000e+06,...,0.775843,0.887922,0.000000,-2.046875,-5.890625,-3.968750,0.221954,3.333333,1.933333,1.933333


# previous

In [48]:
prev = pd.read_csv(path + 'previous_application.csv')
prev.head(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0
6,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
7,1656711,296299,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,7,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
8,2367563,342292,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
9,2579447,334349,Cash loans,NaN,0.0,0.0,NaN,NaN,SATURDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
prev.shape

(1670214, 37)

In [50]:
def previous_applications():
    prev = pd.read_csv(path + 'previous_application.csv')
    
    print("before shape", prev.shape)
    prev, cat_cols = encoder(prev)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    print("after shape", prev_agg.shape)
    return prev_agg

In [51]:
prev = previous_applications()
prev.head(10)

before shape (1670214, 37)
after shape (338857, 246)


,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,REFUSED_HOUR_APPR_PROCESS_START_MAX,REFUSED_HOUR_APPR_PROCESS_START_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000000,24835.5,24835.5,24835.500000,23787.0,23787.0,23787.000000,1.044079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,9251.775,9251.775,9251.775000,179055.0,179055.0,179055.000000,179055.0,179055.0,179055.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,6737.310,98356.995,56553.990000,68809.5,900000.0,435436.500000,68053.5,1035882.0,484191.000000,0.868825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,5357.250,5357.250,5357.250000,24282.0,24282.0,24282.000000,20106.0,20106.0,20106.000000,1.207699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,4813.200,4813.200,4813.200000,0.0,44617.5,22308.750000,0.0,40153.5,20076.750000,1.111173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,2482.920,39954.510,23651.175000,0.0,688500.0,272203.260000,0.0,906615.0,291695.500000,0.759418,...,15.0,15.0,NaN,NaN,NaN,-181.0,-181.0,-181.0,48.0,48.0
100007,1834.290,22678.785,12278.805000,17176.5,247500.0,150530.250000,14616.0,284400.0,166638.750000,0.791139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100008,8019.090,25309.575,15839.696250,0.0,450000.0,155701.800000,0.0,501975.0,162767.700000,0.896459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,7435.845,17341.605,10051.412143,40455.0,110160.0,76741.714286,38574.0,98239.5,70137.642857,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
prev = data_processing(prev)

before process df shape: (338857, 246)
selected dataframe has 246 columns.
There are 77 columns that have missing values.
                                      Missing Count  percent
REFUSED_RATE_DOWN_PAYMENT_MAX                286252     84.5
REFUSED_RATE_DOWN_PAYMENT_MIN                286252     84.5
REFUSED_AMT_DOWN_PAYMENT_MEAN                286252     84.5
REFUSED_AMT_DOWN_PAYMENT_MAX                 286252     84.5
REFUSED_AMT_DOWN_PAYMENT_MIN                 286252     84.5
REFUSED_RATE_DOWN_PAYMENT_MEAN               286252     84.5
REFUSED_APP_CREDIT_PERC_VAR                  280637     82.8
REFUSED_CNT_PAYMENT_MEAN                     230761     68.1
REFUSED_AMT_ANNUITY_MEAN                     230761     68.1
REFUSED_AMT_ANNUITY_MAX                      230761     68.1
REFUSED_AMT_ANNUITY_MIN                      230761     68.1
REFUSED_AMT_GOODS_PRICE_MIN                  228815     67.5
REFUSED_AMT_GOODS_PRICE_MEAN                 228815     67.5
REFUSED_AMT_GOODS_PRICE_

In [53]:
prev.to_csv("C:/data/processing_data/prev.csv")

In [54]:
prev.head(10)

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,APPROVED_HOUR_APPR_PROCESS_START_MAX,APPROVED_HOUR_APPR_PROCESS_START_MEAN,APPROVED_RATE_DOWN_PAYMENT_MIN,APPROVED_RATE_DOWN_PAYMENT_MAX,APPROVED_RATE_DOWN_PAYMENT_MEAN,APPROVED_DAYS_DECISION_MIN,APPROVED_DAYS_DECISION_MAX,APPROVED_DAYS_DECISION_MEAN,APPROVED_CNT_PAYMENT_MEAN,APPROVED_CNT_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000000,24835.5,24835.5,24835.500000,23787.0,23787.0,23787.000000,1.044079,...,13.0,13.000000,0.104326,0.104326,0.104326,-1740.0,-1740.0,-1740.000000,8.000000,8.0
100002,9251.775,9251.775,9251.775000,179055.0,179055.0,179055.000000,179055.0,179055.0,179055.000000,1.000000,...,9.0,9.000000,0.000000,0.000000,0.000000,-606.0,-606.0,-606.000000,24.000000,24.0
100003,6737.310,98356.995,56553.990000,68809.5,900000.0,435436.500000,68053.5,1035882.0,484191.000000,0.868825,...,17.0,14.666667,0.000000,0.100061,0.050030,-2341.0,-746.0,-1305.000000,10.000000,30.0
100004,5357.250,5357.250,5357.250000,24282.0,24282.0,24282.000000,20106.0,20106.0,20106.000000,1.207699,...,5.0,5.000000,0.212008,0.212008,0.212008,-815.0,-815.0,-815.000000,4.000000,4.0
100005,4813.200,4813.200,4813.200000,0.0,44617.5,22308.750000,0.0,40153.5,20076.750000,1.111173,...,11.0,11.000000,0.108964,0.108964,0.108964,-757.0,-757.0,-757.000000,12.000000,12.0
100006,2482.920,39954.510,23651.175000,0.0,688500.0,272203.260000,0.0,906615.0,291695.500000,0.759418,...,15.0,14.400000,0.108994,0.217830,0.163412,-617.0,-181.0,-345.600000,18.000000,90.0
100007,1834.290,22678.785,12278.805000,17176.5,247500.0,150530.250000,14616.0,284400.0,166638.750000,0.791139,...,15.0,12.333333,0.100143,0.218890,0.159516,-2357.0,-374.0,-1222.833333,20.666667,124.0
100008,8019.090,25309.575,15839.696250,0.0,450000.0,155701.800000,0.0,501975.0,162767.700000,0.896459,...,13.0,10.500000,0.000000,0.110243,0.073051,-2536.0,-370.0,-1469.500000,14.000000,56.0
100009,7435.845,17341.605,10051.412143,40455.0,110160.0,76741.714286,38574.0,98239.5,70137.642857,1.000000,...,18.0,13.714286,0.000000,0.209525,0.126602,-1562.0,-74.0,-719.285714,8.000000,56.0


# POS_CASH_BALANCE

In [55]:
pos = pd.read_csv( 'POS_CASH_balance.csv')
pos.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


In [56]:
def pos_cash():
    pos = pd.read_csv(path + 'POS_CASH_balance.csv')
    print("before shape", pos.shape)
    pos, cat_cols = encoder(pos)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    print("after shape", pos_agg.shape)
    return pos_agg

In [57]:
pos = pos_cash()
pos.head(10)

before shape (10001358, 8)
after shape (337252, 18)


,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.000000,0.000000,0.0,0.0,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.000000,0.090909,0.0,0.0,11
100006,-1,-9.619048,21,0,0.000000,0,0.000000,0.857143,0.0,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21
100007,-1,-33.636364,66,0,0.000000,0,0.000000,0.939394,0.0,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66
100008,-2,-43.662651,83,1294,339.060241,0,0.000000,0.939759,0.0,0.0,0.0,0.048193,0.0,0.000000,0.012048,0.0,0.0,83
100009,-1,-33.062500,64,0,0.000000,0,0.000000,0.890625,0.0,0.0,0.0,0.109375,0.0,0.000000,0.000000,0.0,0.0,64


In [58]:
pos = data_processing(pos)

before process df shape: (337252, 18)
selected dataframe has 18 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Count, percent]
Index: []
drop missing values over 60
selected dataframe has 18 columns.
There are 0 columns that have missing values.
after drop missing values df shape: (337252, 18)
the same values feature num : 7
after process df shape: (337252, 11)


In [59]:
pos.head(10)

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.222222,0.000000,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.000000,0.000000,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.071429,0.000000,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.250000,0.000000,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.090909,0.090909,11
100006,-1,-9.619048,21,0,0.000000,0,0.000000,0.857143,0.095238,0.000000,21
100007,-1,-33.636364,66,0,0.000000,0,0.000000,0.939394,0.045455,0.015152,66
100008,-2,-43.662651,83,1294,339.060241,0,0.000000,0.939759,0.048193,0.012048,83
100009,-1,-33.062500,64,0,0.000000,0,0.000000,0.890625,0.109375,0.000000,64


In [60]:
pos.to_csv("C:/data/processing_data/pos.csv")

In [61]:
pos.head(10)

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.222222,0.000000,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.000000,0.000000,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.071429,0.000000,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.250000,0.000000,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.090909,0.090909,11
100006,-1,-9.619048,21,0,0.000000,0,0.000000,0.857143,0.095238,0.000000,21
100007,-1,-33.636364,66,0,0.000000,0,0.000000,0.939394,0.045455,0.015152,66
100008,-2,-43.662651,83,1294,339.060241,0,0.000000,0.939759,0.048193,0.012048,83
100009,-1,-33.062500,64,0,0.000000,0,0.000000,0.890625,0.109375,0.000000,64


# installments payments

In [62]:
def installments_payments():
    ins = pd.read_csv(path + 'installments_payments.csv')
    print("before shape:", ins.shape)
    ins, cat_cols = encoder(ins)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    print("after shape: ",ins_agg.shape)
    return ins_agg

In [63]:
ins = installments_payments()

before shape: (13605401, 8)
after shape:  (339587, 26)


In [64]:
ins = data_processing(ins)

before process df shape: (339587, 26)
selected dataframe has 26 columns.
There are 12 columns that have missing values.
                                Missing Count  percent
INSTAL_PAYMENT_PERC_VAR                   996      0.3
INSTAL_PAYMENT_DIFF_VAR                   977      0.3
INSTAL_PAYMENT_PERC_MEAN                   28      0.0
INSTAL_PAYMENT_PERC_SUM                    19      0.0
INSTAL_PAYMENT_PERC_MAX                     9      0.0
INSTAL_PAYMENT_DIFF_MAX                     9      0.0
INSTAL_PAYMENT_DIFF_MEAN                    9      0.0
INSTAL_AMT_PAYMENT_MIN                      9      0.0
INSTAL_AMT_PAYMENT_MAX                      9      0.0
INSTAL_AMT_PAYMENT_MEAN                     9      0.0
INSTAL_DAYS_ENTRY_PAYMENT_MAX               9      0.0
INSTAL_DAYS_ENTRY_PAYMENT_MEAN              9      0.0
drop missing values over 60
selected dataframe has 26 columns.
There are 12 columns that have missing values.
after drop missing values df shape: (339587, 26)
the sa

In [65]:
ins.to_csv("C:/data/processing_data/ins.csv")

In [66]:
ins.head(10)

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.0,1.000000,7.0,...,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0,7
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.0,1.000000,19.0,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.0,1.000000,25.0,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.0,1.000000,3.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.0,1.000000,9.0,...,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0,9
100006,2,0.0,0.000000,0.0,77.0,19.375000,310.0,1.0,1.000000,16.0,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16
100007,2,12.0,0.954545,63.0,31.0,4.590909,303.0,1.0,0.954545,63.0,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66
100008,2,1317.0,37.628571,1317.0,28.0,11.514286,403.0,1.0,0.971429,34.0,...,27702.964286,969603.750,227.160,432218.295,27360.502714,957617.595,-82.0,-1237.800000,-43323.0,35
100009,1,7.0,0.137255,7.0,23.0,8.725490,445.0,1.0,1.000000,51.0,...,9568.531765,487995.120,6155.280,17341.605,9568.531765,487995.120,-58.0,-864.411765,-44085.0,51


# credit_card_balance :以前信用卡的收支记录, ，每一行是一个月的信用卡余额

In [67]:
cc = pd.read_csv(path + 'credit_card_balance.csv')
cc.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [68]:
def credit_card_balance():
    cc = pd.read_csv(path + 'credit_card_balance.csv')
    print("before shape", cc.shape)
    cc, cat_cols = encoder(cc)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    print("after shape", cc_agg.shape)
    return cc_agg

In [69]:
credit = credit_card_balance() 

before shape (3840312, 23)
after shape (103558, 141)


In [70]:
credit = data_processing(credit)

before process df shape: (103558, 141)
selected dataframe has 141 columns.
There are 49 columns that have missing values.
                                    Missing Count  percent
CC_AMT_PAYMENT_CURRENT_VAR                  31956     30.9
CC_CNT_DRAWINGS_POS_CURRENT_VAR             31866     30.8
CC_AMT_DRAWINGS_OTHER_CURRENT_VAR           31866     30.8
CC_CNT_DRAWINGS_OTHER_CURRENT_VAR           31866     30.8
CC_CNT_DRAWINGS_ATM_CURRENT_VAR             31866     30.8
CC_AMT_DRAWINGS_ATM_CURRENT_VAR             31866     30.8
CC_AMT_DRAWINGS_POS_CURRENT_VAR             31866     30.8
CC_AMT_PAYMENT_CURRENT_MAX                  31438     30.4
CC_AMT_PAYMENT_CURRENT_MIN                  31438     30.4
CC_AMT_PAYMENT_CURRENT_MEAN                 31438     30.4
CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN          31364     30.3
CC_CNT_DRAWINGS_OTHER_CURRENT_MIN           31364     30.3
CC_CNT_DRAWINGS_ATM_CURRENT_MEAN            31364     30.3
CC_CNT_DRAWINGS_ATM_CURRENT_MAX             31364   

In [71]:
credit.to_csv("C:/data/processing_data/credit.csv")